<a href="https://colab.research.google.com/github/onakamari/gRNA_maker/blob/main/v1_gRNA_20210118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Multiple data points
#Request to Ensembl API with multiple ids in mind
import requests, sys

#import a list of accession number and save to a variable
seq_id = "ENSG00000232926" 

#loop should start here!
server = "https://rest.ensembl.org"
ext = "/sequence/id/"+seq_id+"?"

req = requests.get(server+ext, headers={"Content-Type" : "application/json"})

if not req.ok:
  req.raise_for_status()
  sys.exit()

decoded = req.json()
print(repr(decoded))
#save the response to a variable. Move on to the next step.

#Multiple data points
from Bio.Seq import Seq
import os, os.path, json, re
import pandas as pd

path_to_json = 'json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if
              pos_json.endswith('.json')]
#print(json_files)

global data #とりあえず使いやすいのでグローバルに設定 2021/01/18

for index, js in enumerate(json_files):
  with open(os.path.join(path_to_json,js)) as json_file:
    json_text = json.load(json_file)

    #print(json_text)

    id = json_file.name[5:21] #WIP bad index for file name!!
    seq = json_text['seq']
    dotbracket = json_text['dotbracket']
    length = len(seq)
    central_seed = '........'  #Search parameter here. 
    num_ssregions = dotbracket.count(central_seed)  #Number of matching areas.

    data = {'id':id, 'seq':seq, 'num_ssregions':num_ssregions, 'dotbracket':dotbracket, 'length':length}
    print(data)

    list_num_ssregions = data['num_ssregions']

    #add gRNA seq!!
    ss_index = re.search(r'\.\.\.\.\.\.\.\.', data['dotbracket']).span()
    print(ss_index)

    

    """ WIP wanna sort the data by the num_ssregions, but not successful so far 2021/01/18 
    #print(list_num_ssregions)
    #sortby_num_ssregions = sorted(data, key=lambda x:x['num_ssregions'])
    #print(sortby_num_ssregions)
    """
    
    """ 
    #WIP create if else structure to make a better flow of program 
    #Return true if the seq has a satisfying regions.
    matching_regions = central_seed in dotbracket
    print(matching_regions)
    """

    #Search SS regions in dotbracket and return indexes as (a,b).
    import re

    ss_index = re.search(r'\.\.\.\.\.\.\.\.', data['dotbracket']).span()
    #print(ss_index)

    #Find the matching seq and return a complementary seq
    def matching_sequence(x):
      m = Seq(seq[x[0]:x[1]])
      c = m.complement()
      return m,c 

    #print(matching_sequence(ss_index))